In [ ]:
import sys
sys.path.insert(0, '../') # so we can access our config files

import os
import logging
import json
import time

from tqdm.notebook import tqdm
import pandas as pd
import random

# YouTube API Libraries
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from socket import error as SocketError

# Config Imports
from config import youtube

In [ ]:
print(os.environ.get("default_output"))

In [ ]:
youtube_api = build(youtube.YOUTUBE_API_SERVICE_NAME, youtube.YOUTUBE_API_VERSION, developerKey=youtube.YOUTUBE_DATA_API_KEY)

In [ ]:
def keyExists():
    """ Check if we have a key in our environment

    Returns
    -------
    bool
        If we have a key stored in our environment return True 
    """

    return youtube.YOUTUBE_DATA_API_KEY != None

In [ ]:
### Get Video Title, Desc, Tags
def getVideoMetadata(video_id):
    """Download video meta data given an ID

    Method queries the YouTube Data API and retrieves details of the video. These
    details are used as the video desc, title and tags.

    Parameters
    ----------
    video_id : str
        The video_id for the YouTube video, usually found at the end of the URL.

    Returns
    -------
    dict
        A dictionary with the video desc, title and video tags
    """
    try:
        response = (
            youtube_api.videos()
            .list(part="id,snippet,contentDetails", id=video_id)
            .execute()
        )

        # Get Video Details
        try:
            videoContent = response["items"][0]

            if "snippet" in videoContent.keys():
                videoSnippet = videoContent["snippet"]
                print(videoSnippet["title"])
                print(videoSnippet["description"])
                print(videoSnippet["tags"])

                return {
                    "title": videoSnippet["title"],
                    "description": videoSnippet["description"],
                    "tags": videoSnippet["tags"],
                }

        except:
            print("Failed to get title and video")
            return {"title": "", "description": "", "tags": ""}

    except (HttpError, SocketError) as error:
        print(
            "--- HTTP Error occurred while retrieving information for VideoID: {0}. [ERROR]: {1}".format(
                video_id, error
            )
        )

In [ ]:
# Top N comments
def getVideoComments(video_id, num_comments):
    """Download video comments given an ID

    Method queries the YouTube Data API and retrieves the top comments of a video.

    Parameters
    ----------
    video_id : str
        The video_id for the YouTube video, usually found at the end of the URL.
    num_comments: int
        The max number of comments we want from the video.

    Returns
    -------
    dict
        A dictionary with the comments
    """

    counter = num_comments
    nextPageToken = ""
    comments = []

    while counter > 0:
        try:
            # For the first page
            if num_comments == counter:
                response = (
                    youtube_api.commentThreads()
                    .list(part="snippet,replies", videoId=video_id, maxResults=100)
                    .execute()
                )

                # Grab the ID to the next page
                if "nextPageToken" in response.keys():
                    nextPageToken = response["nextPageToken"]

                try:
                    pageComments = []

                    # Get all the comments for the page
                    for item in response["items"]:
                        # Start with the top comment
                        pageComments.append(
                            {
                                item["snippet"]["topLevelComment"]["id"]: item[
                                    "snippet"
                                ]["topLevelComment"]["snippet"]["textOriginal"]
                            }
                        )

                        # If we have replies
                        if "replies" in item.keys():
                            # Get all the replies to the comment
                            pageComments += [
                                {reply["id"]: reply["snippet"]["textOriginal"]}
                                for reply in item["replies"]["comments"]
                            ]

                    # Each page grabs at max 100 comments
                    counter -= 100
                    comments += pageComments

                except:
                    return {"comments": []}

            else:
                # For all subsequent pages
                response = (
                    youtube_api.commentThreads()
                    .list(
                        part="snippet,replies",
                        pageToken=nextPageToken,
                        videoId=video_id,
                        maxResults=100,
                    )
                    .execute()
                )

                # Grab the ID to the next page
                if "nextPageToken" in response.keys():
                    nextPageToken = response["nextPageToken"]

                try:
                    pageComments = []

                    # Get all the comments for the page
                    for item in response["items"]:
                        # Start with the top comment
                        pageComments.append(
                            {
                                item["snippet"]["topLevelComment"]["id"]: item[
                                    "snippet"
                                ]["topLevelComment"]["snippet"]["textOriginal"]
                            }
                        )

                        # If we have replies
                        if "replies" in item.keys():
                            # Get all the replies to the comment
                            pageComments += [
                                {reply["id"]: reply["snippet"]["textOriginal"]}
                                for reply in item["replies"]["comments"]
                            ]

                    # Each page grabs at max 100 comments
                    counter -= 100
                    comments += pageComments

                except:
                    return {"comments": []}

        except (HttpError, SocketError) as error:
            print(
                "--- HTTP Error occurred while retrieving information for VideoID: {0}. [ERROR]: {1}".format(
                    video_id, error
                )
            )

    return {"comments":comments}


In [ ]:
### Get Video Transcript

def getVideoTranscript(video_id, transcriber=None):
    """Download video transcript given an ID

    Method uses the youtube-transcript library to query the API for a video_id. Method cleans 
    returned output and provides a text blob back.
    
    Parameters
    ----------
    video_id : str
        The video_id for the YouTube video, usually found at the end of the URL.
    transcriber : YouTubeTranscriptApi
        Our transcription object. Passed in to prevent constant constructor calls.

    Returns
    -------
    dict
        A dictionary with the raw transcript, cleaned transcript, and pauses in the video.
    """

    # If we did not pass in a transcriber
    if transcriber == None:
        transcriber = YouTubeTranscriptApi()

    try:
        raw_transcript = transcriber.get_transcript(video_id)
    except:
        return {"cleaned_transcript" : "", "raw_transcript" : {}}

    cleaned_transcript = " ".join([phrase['text'] for phrase in raw_transcript])

    return {"cleaned_transcript" : cleaned_transcript, "raw_transcript" : raw_transcript}

In [ ]:
transcriber = YouTubeTranscriptApi()
video_id = '41hCQZAgVa0'

In [ ]:
videoMetaData = getVideoMetadata(video_id)
time.sleep(1)
videoComments = getVideoComments(video_id, 200)
time.sleep(2)
videoTranscript = getVideoTranscript(video_id, transcriber)

In [ ]:
videoMetaData

In [ ]:
response = (
    youtube_api.videos()
    .list(part="id,snippet,contentDetails", id='aUUO7Ajvs1Y')
    .execute()
)

In [ ]:
response["items"]

In [ ]:
import re
import datetime

def parse_duration(duration_string):
    # Define regular expression patterns for hours, minutes, and seconds
    hours_pattern = r'(?P<hours>\d+)H'
    minutes_pattern = r'(?P<minutes>\d+)M'
    seconds_pattern = r'(?P<seconds>\d+)S'

    # Extract the components from the duration string using the regular expressions
    hours = 0
    minutes = 0
    seconds = 0

    hours_match = re.search(hours_pattern, duration_string)
    if hours_match:
        hours = int(hours_match.group('hours'))

    minutes_match = re.search(minutes_pattern, duration_string)
    if minutes_match:
        minutes = int(minutes_match.group('minutes'))

    seconds_match = re.search(seconds_pattern, duration_string)
    if seconds_match:
        seconds = int(seconds_match.group('seconds'))

    # Create a time object with the parsed components
    time_obj = datetime.time(hour=hours, minute=minutes, second=seconds)

    return time_obj

In [ ]:
response["items"][0]

In [ ]:
video = {
    "title": videoMetaData["title"],
    "description": videoMetaData["description"],
    "tags": videoMetaData["tags"],
    "cleaned_transcript": videoTranscript["cleaned_transcript"],
    "raw_transcript": videoTranscript["raw_transcript"], 
    "comments": [list(comment.values())[0] for comment in videoComments["comments"]],
    "comment_ids" : [list(comment.keys())[0] for comment in videoComments["comments"]],
}


In [ ]:
pd.DataFrame(pd.Series(video)).T

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/raw/youtube/videos_0.csv')

In [ ]:
df.head()

In [ ]:
df['title'][0]

In [ ]:
df['tags'][0]

In [ ]:
df['cleaned_transcript'][0]